In [1]:
import numpy as np # used for array operations
import brian2 as b2 # used for neural simulation
from scipy.stats import poisson, binom # used for stats (mean, s.d.)
import matplotlib.pyplot as plt # used for plotting
import pickle # used for saving data
b2.prefs.codegen.target = "numpy" # set Brian2 to use numpy backend

In [98]:
# Simulates multiple independent Poisson processes for a group of neurons
def independent_poisson_processes(num_neurons, rate, time, num_samples): 
    # Convert time to seconds to Brian2 units
    simulation_time = time * b2.second
    # Number of time bins (in milisecs) used to discretize
    n_bins = int(time)
    
    #3D array to store spike trains of parameters
    processes = np.zeros((num_samples, num_neurons, n_bins))

    # tried to vectorize and use for-loops as least as possible, some remain
    for sample in range(num_samples):

        # Create Poisson neurons and monitor their spikes
        poisson_group = b2.PoissonGroup(num_neurons, rate * b2.Hz) # represents the neurons created, firing at the specified rate with int and rate
        spike_monitor = b2.SpikeMonitor(poisson_group) # records the spikes genreated by the poissongroup with source

        # create a network
        net = b2.Network(poisson_group, spike_monitor)
        # run the network
        net.run(simulation_time)
        
        # obtaining spike times for each neuron
        spike_trains = spike_monitor.spike_trains() 

        # Convert continuous spike times to discrete time bins
        spike_indices = np.array([np.floor(t / b2.ms) for t in spike_trains[0] if t < simulation_time])
        
        # Mark spike occurrences in the processes array
        processes[sample, 0, spike_indices.astype(int)] = 1
    
    return processes

In [3]:
def save_data(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)

In [4]:
def load_data(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

In [5]:
# sums across neurons to get population activity
def calculate_population_activity(processes):
    return np.sum(processes, axis=1)

In [6]:
# count total spikes for each neuron in each sample
def calculate_spike_counts(processes):
    return np.sum(processes, axis=2)

In [7]:
# sum spike counts across neurons for each sample
def calculate_total_spike_counts(spike_counts):
    return np.sum(spike_counts, axis=1)

In [8]:
# calculate average spike count across all samples
def calculate_average_spike_count(total_spike_counts):
    return np.mean(total_spike_counts)

In [9]:
# Find neurons with minimum spike count
def find_minimum_spike_count(spike_counts):
    minimum_spike_count = np.min(spike_counts)
    minimum_spike_count_indices = np.where(spike_counts == minimum_spike_count)
    minimum_spike_count_integers = np.arange(1, minimum_spike_count + 1)
    return minimum_spike_count, minimum_spike_count_indices, minimum_spike_count_integers

In [10]:
def correlated_poisson_processes(num_neurons, rate, time, num_common, num_samples):
    # Create a network with independent Poisson neurons
    # Each neuron is reprsented by a seperate PoissonGroup with rate
    net = b2.Network()
    poisson_groups = [b2.PoissonGroup(1, rate*b2.Hz) for _ in range((num_neurons + num_common) * num_samples)]
    net.add(poisson_groups)
    
    # Setting up spike monitors to record the activity of each neuron with a simulation run
    spike_monitors = [b2.SpikeMonitor(group) for group in poisson_groups]
    net.add(spike_monitors)
    net.run(time * b2.ms) # should all of the b2.ms be (t/b2.ms)*10) ????

    # Convert spike trains for Brian2 format to numpy arrays, intialize storing process, convert spike times to discrete time indicies
    spike_trains = [monitor.spike_trains()[0] for monitor in spike_monitors]
    independent_procs = np.zeros((num_samples, num_neurons + num_common, int(time)), dtype=bool)
    spike_indices_list = [[int(t / b2.ms) for t in spike_train if t < time * b2.ms] for spike_train in spike_trains]

    # Ensuring all spike trains have the same length by padding with zeros
    max_length = max(len(indices) for indices in spike_indices_list)
    padded_spike_indices_list = [indices + [0] * (max_length - len(indices)) for indices in spike_indices_list]

    # Convert padded lists to numpy array for efficient processing
    spike_indices = np.array(padded_spike_indices_list)

    # Calculate sample and neuron idicies for efficient array indexing
    sample_indices, neuron_indices = np.divmod(np.arange((num_neurons + num_common) * num_samples), num_neurons + num_common)
    for i, indices in enumerate(spike_indices):
        for index in indices:
            if index < int(time):
                independent_procs[sample_indices[i], neuron_indices[i], index] = True

    # Generate correlated processes:
    correlated_procs = np.zeros((num_samples, num_neurons, int(time)), dtype=bool) #  by combining independent processes
    correlated_procs[:, :num_common, :] = np.cumsum(independent_procs[:, :num_common, :], axis=2) # num_common neurons are correlated through cumsum
    correlated_procs[:, num_common:, :] = independent_procs[:, num_common:num_neurons, :] # remaining neurons maintain their independent firing patterns

    return correlated_procs # should this return more?

In [11]:
def small_network(num_neurons, rate, time):
    neurons = b2.NeuronGroup(num_neurons, 'dv/dt = -v/(10*ms) : 1', threshold='v>1', reset='v=0')
    inputs = b2.PoissonInput(target=neurons, target_var='v', N=num_neurons, rate=rate*b2.Hz, weight=1)
    spike_monitor = b2.SpikeMonitor(neurons)
    net = b2.Network(neurons, inputs, spike_monitor)
    net.run(time * b2.ms)  # Run the network instead of b2.run
    return spike_monitor.spike_trains()

In [12]:
# start - counting functions

# The counting functions are designed to calculate the number of spikes in a neuron or a group of neurons over time.
# These functions take in an array of spike counts and return the cumulative sum of the spike counts up to a specified time.
# The count_at_time function calculates the cumulative sum of the spike counts up to a specified time t,
# while the count1 function calculates the cumulative sum of the spike counts for a single neuron over time.
# The countall function calculates the cumulative sum of the spike counts for all neurons over time,
# and the counting_process_nd function calculates the cumulative sum of the spike counts for multiple neurons over time.

In [13]:
def count_at_time(counts, times, t):
    return np.cumsum(counts[:np.sum(times < t)])

In [14]:
def count1(process, time):
    # **Changed to use np.cumsum** # this is discrete because we need continuous model
    return np.cumsum(process)[:time]

In [15]:
def countall(processes, time):
    counts = []
    for process in processes:
        count = count1(process, time)
        counts.append(count)
    return counts

In [16]:
def counting_process_nd(independent_processes, num_samples, time, num_neurons_to_plot):
    counting_process_nd = [[[0 for _ in range(time)] for _ in range(num_neurons_to_plot)] for _ in range(num_samples)]
    for i in range(num_samples):
        counts = [0] * num_neurons_to_plot
        for j in range(time):
            for k in range(num_neurons_to_plot):
                if independent_processes[i][k][j] == 1:
                    counts[k] += 1
            for k in range(num_neurons_to_plot):
                counting_process_nd[i][k][j] = counts[k]
    return counting_process_nd

In [17]:
# end - counting functions

In [18]:
# start - plot functions

# The plotting functions are designed to visualize the spike counts and other data.
# The plot_neurons_spiking function plots the spike counts for multiple neurons over time,
# while the plot_neuron_spiking_standard_dev function plots the spike counts for a single neuron over time,
# along with the mean and standard deviation of the spike counts.
# The plot_two_neurons_against_time function plots the spike counts for two neurons over time,
# along with the cumulative sum of the spike counts for a third neuron.

In [19]:
def plot_neurons_spiking(processes, time): # showing some examples of neurons spiking
    fig = plt.figure(figsize=(10,6))
    for i in range(len(processes[0])):
        plt.plot(processes[0][i], label=f'Neuron {i}')
    plt.xlabel('Time')
    plt.ylabel('Spike')
    plt.title('Neurons Over Time')
    plt.legend()
    plt.show()

In [20]:
def plot_neuron_spiking_standard_dev(processes, time):
    fig = plt.figure(figsize=(10,6))
    plt.plot(processes[0][0], label='Neuron 1')
    sd = np.std(processes[0][0])
    mean = np.mean(processes[0][0])
    print(f"Mean: {mean:.2f}")
    print(f"Standard Deviation: {sd:.2f}")
    plt.fill_between(range(time), processes[0][0] - sd, processes[0][0] + sd, alpha=0.2, label='Standard Deviation')
    plt.axhline(y=mean, color='black', linestyle='--', label='Mean')
    plt.xlabel('Time')
    plt.ylabel('Spike')
    plt.title('Neuron 1 Over Time with Standard Deviation')
    plt.legend()
    plt.text(0.5, 0.9, f"Mean: {mean:.2f}, SD: {sd:.2f}", transform=plt.gca().transAxes)
    plt.show()

In [21]:
def plot_two_neurons_against_time(processes, time):
    fig = plt.figure(figsize=(10,6))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot(processes[0][0], processes[0][1], np.cumsum(processes[0][2]))
    ax.set_xlabel('Neuron 1')
    ax.set_ylabel('Neuron 2')
    ax.set_zlabel('Cumulative Sum of Neuron 3')
    plt.title('Two Neurons Over Time with Cumulative Sum of Third Neuron')
    plt.show()

In [22]:
def plot_count_neuron1_vs_time(counting_process_nd, num_samples):
    plt.figure(figsize=(10,6))
    for i in range(num_samples):
        plt.plot(counting_process_nd[i][0], label=f'Sample {i}')
    mean_neuron1 = np.mean([counting_process_nd[i][0] for i in range(num_samples)], axis=0)
    plt.plot(mean_neuron1, label='Mean', color='black', linewidth=2)
    plt.xlabel('Time')
    plt.ylabel('Count of Neuron 1')
    plt.title('Count of Neuron 1 Over Time')
    plt.ylim(0, None)  # Set y-axis lower limit to 0
    plt.show()

In [23]:
def plot_count_neuron1_vs_neuron2_vs_time(counting_process_nd, num_samples):
    fig = plt.figure(figsize=(12,8))
    ax = fig.add_subplot(111, projection='3d')
    for i in range(num_samples):
        ax.plot(counting_process_nd[i][0], counting_process_nd[i][1], range(len(counting_process_nd[i][0])))
    mean_neuron1 = np.mean([counting_process_nd[i][0] for i in range(num_samples)], axis=0)
    mean_neuron2 = np.mean([counting_process_nd[i][1] for i in range(num_samples)], axis=0)
    ax.plot(mean_neuron1, mean_neuron2, range(len(mean_neuron1)), color='black', linewidth=2)
    ax.set_xlabel('Count of Neuron 1')
    ax.set_ylabel('Count of Neuron 2')
    ax.set_zlabel('Time', rotation=90)
    ax.set_title('Count of Neuron 1 vs Count of Neuron 2 vs Time')
    ax.set_xlim(0, max([max(counting_process_nd[i][0]) for i in range(num_samples)]))
    ax.set_ylim(0, max([max(counting_process_nd[i][1]) for i in range(num_samples)]))
    ax.set_zlim(0, max([len(counting_process_nd[i][0]) for i in range(num_samples)]))
    plt.show()

In [24]:
def plot_count1(count1_output, title='Count of Spikes Over Time (Single Neuron)'):
    plt.figure(figsize=(10,6))
    plt.plot(count1_output)
    plt.xlabel('Time')
    plt.ylabel('Count')
    plt.title(title)
    plt.show()
    print(f"Plot of {title} generated successfully.")

In [25]:
def plot_vectorized_count1(count1_vectorized_output, title='Count of Spikes Over Time (Single Neuron, Vectorized)'):
    plt.figure(figsize=(10,6))
    plt.plot(count1_vectorized_output)
    plt.xlabel('Time')
    plt.ylabel('Count')
    plt.title(title)
    plt.show()
    print(f"Plot of {title} generated successfully.")

In [26]:
def plot_counts(counts_all, title='Count of Spikes Over Time'):
    plt.figure(figsize=(10,6))
    for i, count in enumerate(counts_all):
        plt.plot(count, label=f'Neuron {i}')
    plt.xlabel('Time')
    plt.ylabel('Count')
    plt.title(title)
    plt.legend()
    plt.show()

In [27]:
def plot_counts_vectorized(counts_all_vectorized, title='Count of Spikes Over Time (Vectorized)'):
    plt.figure(figsize=(10,6))
    for i, count in enumerate(counts_all_vectorized):
        plt.plot(count, label=f'Neuron {i}')
    plt.xlabel('Time')
    plt.ylabel('Count')
    plt.title(title)
    plt.legend()
    plt.show()

In [28]:
def plot_covariance(covariance, time):
    plt.figure(figsize=(10, 6))
    plt.plot(time, covariance)
    plt.xlabel('Time (s)')
    plt.ylabel('Covariance')
    plt.title('Covariance between Neuron Counts 1 and 2 over Time')
    plt.show()

In [29]:
# end - plot functions

In [30]:
# start - stats functions

# The statistical functions are designed to calculate statistical properties of the spike counts.
# These functions take in an array of spike counts and return statistical properties such as the mean, covariance, and slope of the spike counts. 
# The calculate_mean_and_covariance function calculates the mean and covariance of the spike counts for multiple neurons
# over time, while the get_slope function calculates the slope of the spike counts for multiple neurons over time.
# The calculate_mean_with_std function calculates the mean and standard deviation of the spike counts for a single neuron over time,
# and the calculate_covariance_matrix function calculates the covariance matrix of the spike counts for multiple neurons over time.

In [31]:
def calculate_mean_and_covariance(processes):
    means = []
    covariances = []
    for i in range(len(processes[0])):
        mean = np.mean([process[i] for process in processes])
        covariance = np.cov([process[i] for process in processes], rowvar=False)
        means.append(mean)
        covariances.append(covariance)
    return means, covariances

In [32]:
def calculate_covariance(processes):
    num_samples, num_neurons, n_bins = processes.shape
    covariance = np.zeros((n_bins,))

    for i in range(n_bins):
        neuron1_counts = processes[:, 0, i]
        neuron2_counts = processes[:, 1, i]
        covariance[i] = np.cov(neuron1_counts, neuron2_counts)[0, 1]

    return covariance

In [33]:
def calculate_covariance_over_neurons(covariance):
    return np.mean(covariance)

In [34]:
def measure_counts_in_time_windows(processes, time_windows):
    num_samples, num_neurons, n_bins = processes.shape
    mean_counts = []
    covariance_counts = []

    for start, end in time_windows:
        start_idx = int(start * 1000)
        end_idx = int(end * 1000)
        window_processes = processes[:, :, start_idx:end_idx]

        mean_count = np.mean(np.sum(window_processes, axis=2))
        covariance_count = calculate_covariance(window_processes)

        mean_counts.append(mean_count)
        covariance_counts.append(covariance_count)

    return mean_counts, covariance_counts

In [35]:
def simulate_homogenous_poisson_process(rate, simulation_time):
    import numpy as np

    # Generate Poisson process using numpy's random.poisson function
    poisson_process = np.random.poisson(rate * simulation_time)

    return poisson_process

In [36]:
def get_slope(processes):
    slopes = []
    for i in range(len(processes[0])):
        slope = np.polyfit(range(len(processes[0][i])), processes[0][i], 1)[0]
        slopes.append(slope)
    return slopes

In [37]:
def calculate_mean_with_std(processes):
    mean_with_std = np.mean(processes[0][0]) + np.std(processes[0][0])
    return mean_with_std

In [38]:
def calculate_covariance_matrix(processes):
    cov_matrix = np.cov([processes[0][0], processes[0][1]])
    return cov_matrix

In [39]:
# end - stats functions

In [40]:
# start -  Discretize time and sampling functions

# The discretization and sampling functions are designed to discretize the time values into bins of a specified size and sample
# the spike counts at a specified resolution. The discretize_time function discretizes the time values into bins of a specified size,
# while the sample_spikes function samples the spike counts at a specified resolution.
# These functions are useful for analyzing the spike counts at different time scales.

In [41]:
def discretize_time(processes, time):
    discretized_processes = []
    for process in processes:
        discretized_process = np.array([process[i] for i in range(0, len(process), time)])
        discretized_processes.append(discretized_process)
    return discretized_processes

In [42]:
def sample_spikes(processes, resolution):
    sampled_processes = []
    for process in processes:
        sampled_process = process[::resolution]
        sampled_processes.append(sampled_process)
    return sampled_processes

In [43]:
# end -  Discretize time and sampling function

In [44]:
# start - tagging and branding functions

# The tagging and branding functions are designed to tag each event in the spike counts with a Poisson process and simulate
# a network of neurons. The tag_events_with_poisson function tags each event in the spike counts with a Poisson process,
# while the brand_networks function simulates a network of neurons with a specified number of neurons, firing rate, and time,
# and returns the spike trains of the neurons. These functions are useful for analyzing the behavior of neural networks.

In [45]:
def tag_events_with_poisson(processes, rate):
    if isinstance(processes, (np.float64, float, int)):
        # If single value, return single Poisson sample
        return np.random.poisson(rate)
    else:
        # If array, process as before
        tagged_processes = []
        for process in processes:
            tagged_process = np.random.poisson(rate, size=len(process))
            tagged_processes.append(tagged_process)
        return tagged_processes  

In [46]:
def brand_networks(num_neurons, firing_rate, time, num_rates=None):
    neurons = b2.NeuronGroup(num_neurons, 'dv/dt = -v/(10*ms) : 1', threshold='v>1', reset='v=0')

    if num_rates is None:
        # Use a single rate for all neurons
        inputs = b2.PoissonInput(target=neurons, target_var='v', N=num_neurons, rate=firing_rate*b2.Hz, weight=1)
    else:
        # Use multiple rates for different neurons
        input_rates = np.random.uniform(0, firing_rate, size=num_rates)
        inputs = []
        for i in range(num_neurons):
            input_rate = input_rates[i % num_rates] * b2.Hz
            input_ = b2.PoissonInput(target=neurons, target_var='v', N=1, rate=input_rate, weight=1)
            inputs.append(input_)

    spike_monitor = b2.SpikeMonitor(neurons)
    net = b2.Network(neurons, inputs, spike_monitor)
    net.run(time * b2.ms)  # Run the network instead of b2.run
    return spike_monitor.spike_trains() 

In [47]:
# end - tagging and branding functions